# __Projet NLP : Prédiction de note pour des avis de produits Amazon__
## <p style="color:blue;">1- Contexte et description des tâches</p>

Dans le cadre du projet de NLP, nous avons pour objectif de concevoir un système de classification de sentiments appliqué à des avis clients issus d’Amazon. L’objectif principal est de prédire automatiquement la note qu’un utilisateur a attribuée à un produit en se basant uniquement sur le titre et le contenu textuel de l’avis qu’il a rédigé.

<p style="color:red;">A CONTINUER</p>


In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

## <p style="color:blue;">2- Le dataset</p> 
Le dataset utilisé dans ce projet est un CSV qui contient des avis d’utilisateurs Amazon portant sur divers produits. Chaque ligne du fichier est structurée en trois colonnes :

- `label` : la note attribuée par l’utilisateur au produit (de 1 à 5 étoiles),

- `title` : le titre de l’avis,

- `review` : le contenu textuel de l’avis.


In [14]:
train_df = pd.read_csv("../amazon_review_full_csv/train.csv", header=None, names=["label","title","review"], 
                       on_bad_lines='skip',  # pour ignorer les lignes corrompues
                       encoding="utf-8")

test_df = pd.read_csv("../amazon_review_full_csv/test.csv", header=None, names=["label","title","review"], 
                      on_bad_lines='skip', 
                      encoding="utf-8")

In [15]:
#test affichage
train_df.head(5)


,label,title,review
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...


In [16]:
# Distrib des labels (pour l'instant de 1 à 5)
train_df['label'].value_counts()


label
3    600000
5    600000
4    600000
1    600000
2    600000
Name: count, dtype: int64

### <p style="color:#d300ff;">- Objectif de classification</p>
Afin de simplifier la tâche de classification, on a regroupé les notes (qui allaient de 1 à 5) comme suit dans une quatrième colonne `label_3` :

- __négatif__ : notes de 1 ou 2 regroupés en 1,

- __neutre__ : note de 3 regroupés en 2,

- __positif__ : notes de 4 ou 5 regroupés en 3.

Cette transformation permet de formuler une tâche de classification en trois classes correspondant à des niveaux de sentiment.

In [17]:
# Préparation des données
#On va regrouper les labels 1,2 ; 3 ; 4,5 en 3 classes : négatif, neutre, positif
def map_label(x):
    if x in [1,2]:
        return 1  # negatif
    elif x == 3:
        return 2  # neutre
    else:
        return 3  # positif

train_df['label_3'] = train_df['label'].apply(map_label)

train_df['label_3'].value_counts()


label_3
3    1200000
1    1200000
2     600000
Name: count, dtype: int64

Et enfin, pour éviter les erreurs liées aux titres/review vides, on a fusionné les 2 colonnes dans une cinquième nommée `full_review`.

Au final, on ne gardera que `label_3` et `full_review` car les autres nous seront inutiles.

In [18]:
print("Valeurs nulles par colonne:")
print(train_df.isna().sum())

Valeurs nulles par colonne:
label        0
title      188
review       0
label_3      0
dtype: int64


In [21]:
# Fusion des colonnes title et review
train_df['title'] = train_df['title'].fillna('') # Remplacer les valeurs nulles par une chaîne vide
train_df['full_review'] = train_df['title'] + ' ' + train_df['review'] 
train_df.head()

,label,title,review,label_3,full_review
0,3,more like funchuck,Gave this to my dad for a gag gift after direc...,2,more like funchuck Gave this to my dad for a g...
1,5,Inspiring,I hope a lot of people hear this cd. We need m...,3,Inspiring I hope a lot of people hear this cd....
2,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...,3,The best soundtrack ever to anything. I'm read...
3,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...,3,Chrono Cross OST The music of Yasunori Misuda ...
4,5,Too good to be true,Probably the greatest soundtrack in history! U...,3,Too good to be true Probably the greatest soun...


In [22]:
# Suppression des colonnes inutiles
train_df = train_df[['label_3', 'full_review']]
train_df.head()

,label_3,full_review
0,2,more like funchuck Gave this to my dad for a g...
1,3,Inspiring I hope a lot of people hear this cd....
2,3,The best soundtrack ever to anything. I'm read...
3,3,Chrono Cross OST The music of Yasunori Misuda ...
4,3,Too good to be true Probably the greatest soun...


### <p style="color:#d300ff;">- Répartition des données</p>
Pour réaliser le travail on dispose de deux fichiers :

- `train.csv` : le jeu de données d'entraînement qui contient 3 millions d'avis utilisateurs dont 1.2 million négatifs, 1.2 million positifs et 600 000 neutres, 
- `test.csv` : le jeu de données de test qui contient 650 000 d'avis utilisateurs